In [ ]:
# install python dependencies

# %pip install torch torchvision timm pandas geopandas numpy opencv-python scikit-learn matplotlib folium

In [ ]:
# import python packages

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
import torchvision.transforms as T
import timm
import pandas as pd
import geopandas as gpd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
import folium
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from datetime import datetime
import logging
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

In [ ]:
!nvidia-smi

In [ ]:
# clear memory and cache

# del model, optimizer, images, labels, outputs
# del model, optimizer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# set the device for torch backend

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# Structure images, deomgraphic data and labels.

class UrbanRetrofittingDataset(Dataset):
    def __init__(self, excel_file, image_dir, transform=None):
        """
        Args:
            excel_file (str): Path to the Excel file with annotations.
            image_dir (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image pair.
        """
        self.data = pd.read_excel(excel_file, dtype={"Latitude": str, "Longitude": str})
        # maintains precision. Do not change this two step converesion.
        self.data = self.data.astype({"Latitude": np.longdouble, "Longitude": np.longdouble})
        self.image_dir = image_dir
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.data['Change'] = self.label_encoder.fit_transform(self.data['Change'])  # Encode labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        census_block = str(row['ID'])
        lat, lon, direction = row['Latitude'], row['Longitude'], row['Heading']
        start_date, end_date = row['Start'], row['End']
        pop_density_change = row['PopDen_Diff']
        change_percentage = row['PopDen_Diff_Percentage']
        label = row['Change']

        # Load image pair Do not use f{} formatting to add longdouble number
        start_image_path = os.path.join(self.image_dir, census_block,  str(lat)+"_"+str(lon)+"_"+str(direction), f"{pd.to_datetime(start_date).strftime('%b %Y')}.png")
        end_image_path = os.path.join(self.image_dir, census_block, str(lat)+"_"+str(lon)+"_"+str(direction), f"{pd.to_datetime(end_date).strftime('%b %Y')}.png")
        start_image = Image.open(start_image_path).convert('RGB')
        end_image = Image.open(end_image_path).convert('RGB')

        if self.transform:
            start_image = self.transform(start_image)
            end_image = self.transform(end_image)

        # socio-economic data
        socio_economic_data = torch.tensor([pop_density_change, change_percentage], dtype=torch.float32)

        return start_image, end_image, socio_economic_data, label

In [ ]:
# Model architecture

class ViTWithSocioEconomic(nn.Module):
    def __init__(self, num_numeric_features, num_classes):
        super(ViTWithSocioEconomic, self).__init__()
        # Pretrained ViT model from timm library without classification head to get image embeddings.
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
        img_embedding_dim = self.vit.num_features

        # Numeric features embedding
        self.numeric_embed = nn.Sequential(
            nn.Linear(num_numeric_features, 32)
            nn.ReLU(),
            nn.Linear(32, 32)
        )

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(3 * img_embedding_dim + 32, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    # Forward pass
    def forward(self, start_image, end_image, numeric_feats):
        start_feat = self.vit(start_image)
        end_feat = self.vit(end_image)
        diff_feat = end_feat - start_feat
        numeric_feat = self.numeric_embed(numeric_feats)
        combined = torch.cat([start_feat, end_feat, diff_feat, numeric_feat], dim=1)
        return self.classifier(combined)

In [ ]:
# Training function with per-class accuracy and loss tracking

def train_epoch(model, loader, criterion, optimizer, device, num_classes):
    model.train()
    total_samples = 0
    running_loss = 0.0
    correct_total = 0
    
    class_correct = [0] * num_classes
    class_total = [0] * num_classes
    class_losses = [0.0] * num_classes
    
    for start_imgs, end_imgs, socio_econ, labels in loader:
        start_imgs, end_imgs = start_imgs.to(device), end_imgs.to(device)
        socio_econ, labels = socio_econ.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(start_imgs, end_imgs, socio_econ)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_size = labels.size(0)
        running_loss += loss.item() * batch_size
        total_samples += batch_size

        preds = outputs.argmax(dim=1)
        correct_total += (preds == labels).sum().item()

        for cls in range(num_classes):
            cls_mask = labels == cls
            if cls_mask.sum() > 0:
                cls_outputs = outputs[cls_mask]
                cls_labels = labels[cls_mask]
                class_losses[cls] += criterion(cls_outputs, cls_labels).item() * cls_mask.sum().item()
                class_correct[cls] += (cls_outputs.argmax(dim=1) == cls_labels).sum().item()
                class_total[cls] += cls_mask.sum().item()

    epoch_loss = running_loss / total_samples
    overall_acc = 100 * correct_total / total_samples

    class_acc = [100 * class_correct[i] / class_total[i] if class_total[i] else 0 for i in range(num_classes)]
    class_avg_loss = [class_losses[i] / class_total[i] if class_total[i] else 0 for i in range(num_classes)]

    return epoch_loss, overall_acc, class_acc, class_avg_loss

In [ ]:
# Validation function with top-2 accuracy and per-class metrics

def validate_epoch(model, loader, criterion, device, num_classes):
    model.eval()
    total_samples = 0
    running_loss = 0.0
    correct_total = 0

    class_correct = [0] * num_classes
    class_total = [0] * num_classes
    class_losses = [0.0] * num_classes
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for start_imgs, end_imgs, socio_econ, labels in loader:
            start_imgs, end_imgs = start_imgs.to(device), end_imgs.to(device)
            socio_econ, labels = socio_econ.to(device), labels.to(device)

            outputs = model(start_imgs, end_imgs, socio_econ)
            loss = criterion(outputs, labels)

            batch_size = labels.size(0)
            running_loss += loss.item() * batch_size
            total_samples += batch_size

            _, preds_top2 = outputs.topk(2, dim=1)
            correct_top2 = preds_top2.eq(labels.unsqueeze(1)).any(dim=1)
            stored_pred_batch = torch.where(correct_top2, labels, preds_top2[:, 0])

            all_preds.extend(stored_pred_batch.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct_total += correct_top2.sum().item()

            for cls in range(num_classes):
                cls_mask = labels == cls
                if cls_mask.sum() > 0:
                    cls_outputs = outputs[cls_mask]
                    cls_labels = labels[cls_mask]
                    class_losses[cls] += criterion(cls_outputs, cls_labels).item() * cls_mask.sum().item()
                    class_correct[cls] += correct_top2[cls_mask].sum().item()
                    class_total[cls] += cls_mask.sum().item()


    epoch_loss = running_loss / total_samples
    overall_acc = 100 * correct_total / total_samples

    class_acc = [100 * class_correct[i] / class_total[i] if class_total[i] else 0 for i in range(num_classes)]
    class_avg_loss = [class_losses[i] / class_total[i] if class_total[i] else 0 for i in range(num_classes)]

    return epoch_loss, overall_acc, class_acc, class_avg_loss, all_preds, all_labels

In [ ]:
# create a directory for saving and loading checkpoints.
checkpoint_dir = "ViT_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
MODEL_NAME = "temporal_vit_spatial_stratified_vali_top2_per_weighted_class_loss_v21"
CHECKPOINT_PATH = f"{checkpoint_dir}/{MODEL_NAME}_epoch_30.pth" 
# directory for storing logs
logs_dir = "logs"
os.makedirs(logs_dir, exist_ok=True)
LOG_FILE = f"{logs_dir}/{MODEL_NAME}.txt"
GRAPH_PATH = f"Model Graphs/{MODEL_NAME}.png"

# Hyperparameters
BATCH_SIZE = 256
NUM_EPOCHS = 30
LEARNING_RATE = 0.0001
WEIGHT_DECAY_RATE = 0.00001
NUM_CLASSES = 5  # Five aspects of urban retrofitting

# Dataset and DataLoader
excel_File = "every_10_nochange_recapital_doubled_2013_2023_pop_den_labels.xlsx"
image_dir = "Streetview_data"

In [ ]:
# Configure logging to file and console

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        # Append mode
        logging.FileHandler(LOG_FILE, mode='a'),
        # Also print to console
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
# Data Loading

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Data transformations

train_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Spatial stratified sampling split function 

def spatial_stratified_split(dataset, val_ratio=0.2, n_clusters=10):
    """
    Perform spatial stratified sampling split and return PyTorch-compatible UrbanRetrofittingDataset subsets.
    NOTE: n_clusters = 4 included at least 1 sample per class in each cluster. 
    Args:
        df (pd.DataFrame): Raw dataframe loaded from Excel.
        image_dir (str): Path to the image directory.
        transform (callable): Torchvision transform for images.
        val_ratio (float): Ratio for validation split.
        n_clusters (int): Number of spatial clusters.

    Returns:
        train_dataset (Subset), val_dataset (Subset), label_encoder
    """
    df = dataset.data.copy()
    
    # Ensure correct types
    df = df.dropna(subset=['Latitude', 'Longitude'])

    # Spatial clustering
    coords = df[['Latitude', 'Longitude']].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Cluster'] = kmeans.fit_predict(coords)

    # Stratified sampling within each spatial cluster
    train_indices, val_indices = [], []

    for cluster in df['Cluster'].unique():
        cluster_df = df[df['Cluster'] == cluster]
        for label in cluster_df['Change'].unique():
            label_df = cluster_df[cluster_df['Change'] == label]
            n_val = max(1, int(len(label_df) * val_ratio))
            val_sample = label_df.sample(n=n_val, random_state=42)
            train_sample = label_df.drop(val_sample.index)
            train_indices.extend(train_sample.index)
            val_indices.extend(val_sample.index)

    # Reset index to use Subset
    df.reset_index(drop=True, inplace=True)

    dataset.data = df

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs='EPSG:4326')

    return train_indices, val_indices, gdf

logger.info("Data Loading Initiated!")
# initializing dataset twice to use multiple transforms for training and validation.
train_dataset = UrbanRetrofittingDataset(excel_File, image_dir, train_transform)
val_dataset = UrbanRetrofittingDataset(excel_File, image_dir, val_transform)

# use any one to get train and val indices.
train_indices, val_indices, clusters_gdf = spatial_stratified_split(dataset=train_dataset, val_ratio=0.2, n_clusters=20)
_,_,_ = spatial_stratified_split(dataset=val_dataset, val_ratio=0.2, n_clusters=20)

train_dataset = Subset(train_dataset, train_indices)
val_dataset = Subset(val_dataset, val_indices)
logger.info("Split data into training and validation.")


In [ ]:
# Log cluster distribution

all_labels = train_dataset.dataset.data['Change']

def log_distribution(all_labels, dataset, name):
    label_counts = Counter([all_labels[i] for i in dataset.indices])
    label_encoder = dataset.dataset.label_encoder
    logger.info(f"{name} distribution:")
    for label_idx in sorted(label_counts):
        logger.info(f"\t{label_encoder.classes_[label_idx]}: {label_counts[label_idx]}")

log_distribution(all_labels, train_dataset, "Train")
log_distribution(all_labels, val_dataset, "Validation")

In [ ]:
# Summary of clusters and class distribution for training set

train_df = train_dataset.dataset.data.loc[train_indices]
counts = train_df.groupby(['Cluster', 'Change']).size().unstack(fill_value=0)
logger.info("Training Sample counts per class in each cluster:")
logger.info(counts)

In [ ]:
# Summary of clusters and class distribution for validation set

val_df = val_dataset.dataset.data.loc[val_indices]
counts = val_df.groupby(['Cluster', 'Change']).size().unstack(fill_value=0)
logger.info("Validation Sample counts per class in each cluster:")
logger.info(counts)

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Prepare colors for classes
classes = train_dataset.dataset.label_encoder.classes_
num_classes = len(classes)
colormap = cm.get_cmap('tab10', num_classes)
class_colors = {classes[i]: mcolors.rgb2hex(colormap(i)) for i in range(num_classes)}

# Get cluster polygons (convex hull)
cluster_polys = clusters_gdf[['Cluster', 'geometry']].dissolve(by='Cluster', as_index=False, aggfunc='first')
cluster_polys['geometry'] = cluster_polys.geometry.convex_hull
cluster_polys = gpd.GeoDataFrame(cluster_polys, geometry="geometry", crs='EPSG:4326')
cluster_polys.rename(columns={0: 'geometry'}, inplace=True)

In [ ]:
# Function to visualize clusters and samples using folium map

def create_folium_cluster_map(data_gdf, class_colors, title):
    m = folium.Map(location=[data_gdf['Latitude'].mean(), data_gdf['Longitude'].mean()], zoom_start=11)
    
    # Plot clusters (polygons)
    for _, row in cluster_polys.iterrows():
        folium.GeoJson(
            row['geometry'],
            style_function=lambda feature, clr=row['Cluster']: {
                'fillColor': f"#cccccc",
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.8
            },
            tooltip=f"Cluster {row['Cluster']}"
        ).add_to(m)

    # Add class samples as points
    for _, row in data_gdf.iterrows():
        cls = row['Change']
        color = class_colors[classes[cls]]
        popup = f"Class: {cls}<br>Cluster: {row['Cluster']}"
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=popup
        ).add_to(m)

    # Add legend
    legend_html = "<div style='position: fixed; bottom: 20px; left: 20px; z-index: 9999; background-color: white; padding: 10px; border: 1px solid #ccc;'>"
    legend_html += "<b>Class Colors</b><br>"
    for cls, color in class_colors.items():
        legend_html += f"<i style='background:{color};width:10px;height:10px;display:inline-block;'></i> {cls}<br>"
    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

# Generate and save maps
train_map = create_folium_cluster_map(train_df, class_colors, "Training Data Map")
val_map = create_folium_cluster_map(val_df, class_colors, "Validation Data Map")

In [ ]:
# visualize training set map.
train_map

In [ ]:
# visualize validation set map. 
val_map

In [ ]:
# display traing set dataframe.
train_df

In [ ]:
# display validation set dataframe.
val_df

In [ ]:
# Extract labels from training dataset
# train_labels = [sample[3] for sample in train_dataset]  # sample[3] is the label in __getitem__
all_labels = train_dataset.dataset.data['Change'] #defined above already
train_labels = [all_labels[i] for i in train_dataset.indices]
num_classes = len(set(train_labels))

# Compute class sample weights
# class_counts = np.array([train_labels.count(i) for i in range(num_classes)])
# class_weights_sampler = 1. / class_counts

# Class sample weights calculated manually to avoid extreme weights for very rare classes.
class_weights_sampler = np.array([0.0036, 0.0034, 0.0150, 0.0050, 0.0060]) # v21
sample_weights = np.array([class_weights_sampler[label] for label in train_labels])

In [ ]:
# Weighted sampler for drawing training samples.
sampler = WeightedRandomSampler(weights=sample_weights,
                                 num_samples=len(sample_weights),
                                 replacement=True)

In [ ]:

# Compute class weights for loss function.
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(labels),
#     y=labels
# )

# Manually calculated class weights to avoid extreme weights for very rare classes.
class_weights = np.array([1.66, 7.22, 5.56, 9.22, 9.34]) #v21

weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
logger.info("Class weights used for loss function:")
for label, weight in zip(train_dataset.dataset.label_encoder.classes_, class_weights):
    logger.info(f"\tClass {label}: Weight={weight:.4f}")

In [ ]:
# to enable faster data loading using multiple workers for data loading.
import multiprocessing
num_workers = multiprocessing.cpu_count() // 2
num_workers
# Above result was 48

In [ ]:
# Data Loaders.

logger.info(f"Batch Size: {BATCH_SIZE}, Workers: {num_workers}, pin_memory: True, LR: {LEARNING_RATE}, Wt. Decay: {WEIGHT_DECAY_RATE}")# this will be a lazy loader. Just initialization happens here. Real work happens inside for loop of data loader.

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=num_workers, pin_memory=True)
logger.info("Training Loader is now ready!")

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
logger.info("Validation Loader is now ready!")

In [ ]:
# Inference

# Model, Loss, Optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTWithSocioEconomic(2, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY_RATE)

# Load checkpoint if exists
if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    train_acc_per_class = checkpoint['train_acc_per_class']
    val_acc_per_class = checkpoint['val_acc_per_class']
    train_loss_per_class = checkpoint['train_loss_per_class']
    val_loss_per_class = checkpoint['val_loss_per_class']
    overall_train_acc = checkpoint['overall_train_acc']
    overall_val_acc = checkpoint['overall_val_acc']
    overall_train_loss = checkpoint['overall_train_loss']
    overall_val_loss = checkpoint['overall_val_loss']
    
    val_preds = checkpoint['val_preds']
    val_labels = checkpoint['val_labels']
    
    logger.info(f"Loaded checkpoint from epoch {start_epoch}")
else:
    # Initialize lists to store losses and accuracies
    start_epoch = 0
    train_acc_per_class = []
    val_acc_per_class = []
    train_loss_per_class = []
    val_loss_per_class = []
    overall_train_acc = []
    overall_val_acc = []
    overall_train_loss = []
    overall_val_loss = []
    val_preds = None
    val_labels = None

    logger.info("No checkpoint found - starting new training")

for epoch in range(start_epoch, NUM_EPOCHS):
    train_loss, train_overall_acc,  train_class_acc, train_class_loss = train_epoch(model, train_loader, criterion, optimizer, device, NUM_CLASSES)
    val_loss, val_overall_acc, val_class_acc, val_class_loss, val_preds, val_labels  = validate_epoch(model, val_loader, criterion, device, NUM_CLASSES)
    
    train_acc_per_class.append(train_class_acc)
    val_acc_per_class.append(val_class_acc)
    train_loss_per_class.append(train_class_loss)
    val_loss_per_class.append(val_class_loss)
    overall_train_acc.append(train_overall_acc)
    overall_val_acc.append(val_overall_acc)
    overall_train_loss.append(train_loss)
    overall_val_loss.append(val_loss)

    logger.info(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    logger.info(f"Train Accuracy: {train_overall_acc:.2f}%")
    logger.info(f"Train Loss: {train_loss:.4f}")
    for idx, cls in enumerate(train_dataset.dataset.label_encoder.classes_):
        logger.info(f"\tTrain Class '{cls}': Acc={train_class_acc[idx]:.2f}%, Loss={train_class_loss[idx]:.4f}")
    
    logger.info(f"Validation Accuracy: {val_overall_acc:.2f}%")
    logger.info(f"Validation Loss: {val_loss:.4f}")
    for idx, cls in enumerate(train_dataset.dataset.label_encoder.classes_):
        logger.info(f"\tVal Class '{cls}': Acc={val_class_acc[idx]:.2f}%, Loss={val_class_loss[idx]:.4f}")

    # Save checkpoints every 2 epochs
    # if (epoch + 1) % 2 == 0:
    if (epoch + 1) >= 18:
        CHECKPOINT_PATH = os.path.join(checkpoint_dir, f"temporal_vit_spatial_stratified_vali_top2_per_weighted_class_loss_v21_epoch_{epoch+1}.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_acc_per_class': train_acc_per_class,
            'val_acc_per_class': val_acc_per_class,
            'train_loss_per_class': train_loss_per_class,
            'val_loss_per_class': val_loss_per_class,
            'overall_train_acc':overall_train_acc,
            'overall_val_acc':overall_val_acc,
            'overall_train_loss':overall_train_loss,
            'overall_val_loss':overall_val_loss,
            'val_preds': val_preds,
            'val_labels': val_labels
    
        }, CHECKPOINT_PATH)
        logger.info(f"Saved checkpoint to {CHECKPOINT_PATH}")

    logger.info("-"*100)

In [ ]:
# Classification Report
classes = val_dataset.dataset.label_encoder.classes_
print(classification_report(val_labels, val_preds, target_names=classes, digits=4))

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# visualize accuracy graphs.

import matplotlib.pyplot as plt

# Define class labels
classes = ['No Change', 'Re-building', 'Re-capital', 'Re-inhabitation', 'Re-transportation']

color_map = {
    'No Change': "#c2bf04",         # gold
    'Re-building': "#ff00ff",       # darkorange
    'Re-capital': "#005E05",        # magneta
    'Re-inhabitation': '#00bfff',   # red
    'Re-transportation': '#ff0000'  # deepskyblue
}

# Epochs
epochs = list(range(1, len(train_acc_per_class) + 1))

# Helper to convert list of lists to class-wise dict
def transpose_to_class_dict(data):
    return {cls: [epoch_data[i] for epoch_data in data] for i, cls in enumerate(classes)}

# Convert input lists to class-wise dicts
train_acc_class = transpose_to_class_dict(train_acc_per_class)
val_acc_class = transpose_to_class_dict(val_acc_per_class)
train_loss_class = transpose_to_class_dict(train_loss_per_class)
val_loss_class = transpose_to_class_dict(val_loss_per_class)

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 10))

# Training Accuracy per Class
for cls in classes:
    axs[0].plot(epochs, train_acc_class[cls], label=cls, color=color_map[cls])
axs[0].set_title('Training Accuracy per Category')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy (%)')
axs[0].set_xticks(epochs)
axs[0].set_xlim(1,30)
axs[0].set_ylim(20,100)
# axs[0].set_yticks(list(range(0,101,10))) 
axs[0].legend(loc='lower right')
axs[0].legend()

# Validation Accuracy per Class
for cls in classes:
    axs[1].plot(epochs, val_acc_class[cls], label=cls, color=color_map[cls])
axs[1].set_title('Validation Accuracy per Category')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].set_xticks(epochs)
axs[1].set_xlim(1,30)
axs[1].set_ylim(20,100)
axs[1].legend()

# Overall Accuracy
axs[2].plot(epochs, overall_train_acc, label='Training')
axs[2].plot(epochs, overall_val_acc, label='Validation')
axs[2].set_title('Overall Accuracy')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('Accuracy (%)')
axs[2].set_xticks(epochs)
axs[2].set_xlim(1,30)
axs[2].set_ylim(20,100)
axs[2].legend()

# turn the grid on for every subplot
for ax in axs.ravel():
    ax.grid(True)
    ax.legend(loc='lower right')
plt.tight_layout()
# plt.savefig(GRAPH_PATH)
plt.show()

In [ ]:
# visualize predictions of the model

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

model = ViTWithSocioEconomic(2, NUM_CLASSES).to(device)
CHECKPOINT_PATH = "ViT_checkpoints/temporal_vit_spatial_stratified_vali_top2_per_weighted_class_loss_v21_epoch_30.pth"
checkpoint = torch.load(CHECKPOINT_PATH, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])

# Re-run validation to get all outputs
model.eval()
all_preds = []
all_labels = []
all_probs = []
all_paths = []

with torch.no_grad():
    for batch in val_loader:
        start_imgs, end_imgs, socio_econ, labels = batch
        start_imgs, end_imgs = start_imgs.to(device), end_imgs.to(device)
        socio_econ, labels = socio_econ.to(device), labels.to(device)
        
        outputs = model(start_imgs, end_imgs, socio_econ)
        probs = torch.softmax(outputs, dim=1)
        top_probs, preds = torch.max(probs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(top_probs.cpu().numpy())

        # Save for visualization
        for i in range(len(labels)):
            sample = val_dataset[i]  # since it's a Subset
            metadata = val_dataset.dataset.data.iloc[val_dataset.indices[i]]
            all_paths.append({
                'start_img': start_imgs[i].cpu(),
                'end_img': end_imgs[i].cpu(),
                'label': labels[i].item(),
                'pred': preds[i].item(),
                'confidence': top_probs[i].item(),
                'census_block': metadata['ID'],
                'lat': metadata['Latitude'],
                'lon': metadata['Longitude'],
                'heading': metadata['Heading'],
                'start_date': metadata['Start'],
                'end_date': metadata['End']
            })

# Organize samples by class
class_samples = {i: {'correct': [], 'incorrect': []} for i in range(NUM_CLASSES)}

for info in all_paths:
    cls = info['label']
    if info['pred'] == info['label']:
        if len(class_samples[cls]['correct']) < 1:
            class_samples[cls]['correct'].append(info)
    else:
        if len(class_samples[cls]['incorrect']) < 1:
            class_samples[cls]['incorrect'].append(info)

# Define label map
label_map = dict(enumerate(val_dataset.dataset.label_encoder.classes_))

# Plotting function
def plot_sample(info):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    for i, (img, label) in enumerate(zip([info['start_img'], info['end_img']], ['Start', 'End'])):
        img = img.permute(1, 2, 0).numpy()
        img = img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # Unnormalize
        img = np.clip(img, 0, 1)
        axes[i].imshow(img)
        axes[i].set_title(f"{label}: {info['start_date'] if label=='Start' else info['end_date']}")
        axes[i].axis('off')

    plt.suptitle(
        f"Predicted: {label_map[info['pred']]} ({info['confidence']*100:.2f}%) | Actual: {label_map[info['label']]}\nCensus Block: {info['census_block']} | Location: {info['lat']}, {info['lon']} | Heading: {info['heading']}",
        color='green' if info['pred'] == info['label'] else 'red',
        fontsize=10
    )
    plt.tight_layout()
    plt.show()

# Display 2 per class (correct + incorrect)
for cls in range(NUM_CLASSES):
    if class_samples[cls]['correct']:
        plot_sample(class_samples[cls]['correct'][0])
    if class_samples[cls]['incorrect']:
        plot_sample(class_samples[cls]['incorrect'][0])


In [ ]:
# Store model's prediction for full dataset to an excel file from which accracy for each category across census blocks group can be calculated.

# Data Loading
# Dataset and DataLoader
excel_File = "every_10_nochange_recapital_doubled_2013_2023_pop_den_labels.xlsx"
image_dir = "Streetview_data"
BATCH_SIZE = 256
NUM_CLASSES = 5
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load entire dataset first with val_transform (no augmentation, just resizing and normalization)
full_dataset = UrbanRetrofittingDataset(excel_File, image_dir, val_transform)

# Get only validation subset using original val_indices
val_subset = Subset(full_dataset, val_indices)

data_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

model = ViTWithSocioEconomic(2, NUM_CLASSES).to(device)
CHECKPOINT_PATH = "ViT_checkpoints/temporal_vit_spatial_stratified_vali_top2_per_weighted_class_loss_v21_epoch_30.pth"
checkpoint = torch.load(CHECKPOINT_PATH, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])

# Set model to eval mode
model.eval()

# Collect predictions and true labels
all_preds = []
all_true_labels = []

with torch.no_grad():
    for start_imgs, end_imgs, socio_econ, labels in data_loader:
        start_imgs = start_imgs.to(device)
        end_imgs = end_imgs.to(device)
        socio_econ = socio_econ.to(device)

        outputs = model(start_imgs, end_imgs, socio_econ)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Reload original Excel (to retain all original columns)
df = pd.read_excel(excel_File, dtype={"Latitude": str, "Longitude": str})
df = df.astype({"Latitude": np.longdouble, "Longitude": np.longdouble})

# Fit label encoder again to get label mappings
label_encoder = LabelEncoder()
df['Change_Encoded'] = label_encoder.fit_transform(df['Change'])

# Add predictions
df['Prediction'] = all_preds
df['Prediction_Label'] = label_encoder.inverse_transform(df['Prediction'].astype(int))

# Save to new Excel
output_excel = "val_dataset_predictions_for_mapping.xlsx"
df.to_excel(output_excel, index=False)

print(f"✅ Predictions for entire dataset saved to: {output_excel}")


In [ ]:
# Now, calculate accuracy for each category across census blocks group and merge with shapefile for model accuracy visualization.

import pandas as pd
import geopandas as gpd
import numpy as np

# Load files
excel_file = "val_dataset_predictions_for_mapping.xlsx"
shp_file = "Shapefile/mecklengburg_county_census_block_group_2013.shp"
output_shp = "Shapefile/latest_accuracy_per_census_block_group.shp"

# Load Excel data
df = pd.read_excel(excel_file)

# Ensure ID is string for merge
df['ID'] = df['ID'].astype(str)

# List all unique true change categories in the dataset
all_categories = df['Change'].dropna().unique()

# Function to compute accuracy metrics per Census Block Group
def compute_accuracy(group):
    total = len(group)
    correct = (group['Change'] == group['Prediction_Label']).sum()
    overall_acc = round((correct / total) * 100, 2) if total else np.nan
    result = {'overall_accuracy': overall_acc}

    # Compute accuracy per category
    for category in all_categories:
        cat_group = group[group['Change'] == category]
        cat_total = len(cat_group)
        if cat_total:
            correct_cat = (cat_group['Change'] == cat_group['Prediction_Label']).sum()
            cat_acc = round((correct_cat / cat_total) * 100, 2)
        else:
            cat_acc = np.nan
        result[f'acc_{category}'] = cat_acc

    return pd.Series(result)

# Apply the accuracy function per Census Block Group
accuracy_by_id = df.groupby('ID').apply(compute_accuracy).reset_index()

# Load the shapefile
gdf = gpd.read_file(shp_file)

# Ensure GEOID is string for merging
gdf['GEOID'] = gdf['GEOID'].astype(str)

# Merge shapefile with accuracy data
merged_gdf = gdf.merge(accuracy_by_id, how='left', left_on='GEOID', right_on='ID')

# Identify accuracy columns (those starting with 'acc_' or 'overall_accuracy')
accuracy_cols = [col for col in merged_gdf.columns if col.startswith('acc_') or col == 'overall_accuracy']
print(accuracy_cols)
# Replace NaN with -9999 in only those columns
merged_gdf[accuracy_cols] = merged_gdf[accuracy_cols].fillna(-9999)

# Export the merged shapefile.
merged_gdf.to_file(output_shp)